### NLP + SQL

#### SET DATABASE

In [3]:
import sqlite3

# Conectar a la base de datos
conn = sqlite3.connect('Chinook.db')
cursor = conn.cursor()

In [4]:
with open('E:/Repos/IA/NLP-SQL/Chinook.sql', 'r', encoding='utf-8') as file:
    sql_script = file.read()

In [5]:
cursor.executescript(sql_script)
conn.commit()

#### DATABASE TEST

In [1]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

#### Inicio de servicio vertaxai + configuracion.

In [3]:
import vertexai
import os

vertexai.init(
    project="gen-lang-client-0730997933",
    location="us-central1"
)

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'gen-lang-client-0730997933-9e99e8511560.json'

#### Modelos

In [4]:
from dotenv import load_dotenv
import os
from langchain_google_vertexai import ChatVertexAI

load_dotenv()

GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

llm = ChatVertexAI(model="gemini-1.5-pro-002")


#### Chain NL -> SQL

In [5]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "las las canciones AC/DC agrupadas por album y el total de canciones en cada uno de ellos, dame solo la consulta como tal sin markdown ni otro ccontenido agregado."})


In [6]:
response


'SELECT "T"."AlbumId", COUNT(*) FROM "Track" AS "T" INNER JOIN "Album" AS "A" ON "T"."AlbumId" = "A"."AlbumId" INNER JOIN "Artist" AS "AR" ON "A"."ArtistId" = "AR"."ArtistId" WHERE "AR"."Name" = \'AC/DC\' GROUP BY "T"."AlbumId"'

In [7]:
db.run(response)

'[(1, 10), (4, 8)]'

In [8]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query
chain.invoke({"question": "las canciones AC/DC agrupadas por album y el total de canciones en cada uno de ellos, dame solo la consulta como tal sin markdown ni otro ccontenido agregado."})

'[(1, 10), (4, 8)]'

#### Agent

In [9]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [10]:
agent_executor.invoke(
    "List the total sales per country. Which country's customers spent the most?"
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Invoice'}`
responded: The Invoice table sounds like it could have sales information.  Let me check the schema.


CREATE TABLE "Invoice" (
	"InvoiceId" INTEGER NOT NULL, 
	"CustomerId" INTEGER NOT NULL, 
	"InvoiceDate" DATETIME NOT NULL, 
	"BillingAddress" NVARCHAR(70), 
	"BillingCity" NVARCHAR(40), 
	"BillingState" NVARCHAR(40), 
	"BillingCountry" NVARCHAR(40), 
	"BillingPostalCode" NVARCHAR(10), 
	"Total" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("InvoiceId"), 
	FOREIGN KEY("CustomerId") REFERENCES "Customer" ("CustomerId")
)

/*
3 rows from Invoice table:
InvoiceId	CustomerId	InvoiceDate	BillingAddress	BillingCity	BillingState	BillingCountry	BillingPostalCode	Total
1	2	2021-01-01 00:00:00	Theodor-Heuss-Straße 34	Stuttgart	None	Germany	70174	1.98


Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ResourceExhausted: 429 Online prediction request quota exceeded for gemini-1.5-pro. Please try again later with backoff..
Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ResourceExhausted: 429 Online prediction request quota exceeded for gemini-1.5-pro. Please try again later with backoff..
Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ResourceExhausted: 429 Online prediction request quota exceeded for gemini-1.5-pro. Please try again later with backoff..
Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 8.0 seconds as it raised ResourceExhausted: 429 Online prediction request quota exceeded for gemini-1.5-pro. Please


Invoking: `sql_db_query` with `{'query': 'SELECT BillingCountry, SUM(Total) AS TotalSales FROM Invoice GROUP BY BillingCountry ORDER BY TotalSales DESC LIMIT 10'}`


[('USA', 523.06), ('Canada', 303.96), ('France', 195.1), ('Brazil', 190.1), ('Germany', 156.48), ('United Kingdom', 112.86), ('Czech Republic', 90.24), ('Portugal', 77.24), ('India', 75.26), ('Chile', 46.62)]Here are the total sales per country, ordered from most to least:
USA: $523.06
Canada: $303.96
France: $195.10
Brazil: $190.10
Germany: $156.48
United Kingdom: $112.86
Czech Republic: $90.24
Portugal: $77.24
India: $75.26
Chile: $46.62

Customers from the USA spent the most.


> Finished chain.


{'input': "List the total sales per country. Which country's customers spent the most?",
 'output': 'Here are the total sales per country, ordered from most to least:\nUSA: $523.06\nCanada: $303.96\nFrance: $195.10\nBrazil: $190.10\nGermany: $156.48\nUnited Kingdom: $112.86\nCzech Republic: $90.24\nPortugal: $77.24\nIndia: $75.26\nChile: $46.62\n\nCustomers from the USA spent the most.\n'}

#### Examples

In [11]:
examples = [
    {"input": "List all artists.", "query": "SELECT * FROM Artist;"},
    {
        "input": "Find all albums for the artist 'AC/DC'.",
        "query": "SELECT * FROM Album WHERE ArtistId = (SELECT ArtistId FROM Artist WHERE Name = 'AC/DC');",
    },
    {
        "input": "List all tracks in the 'Rock' genre.",
        "query": "SELECT * FROM Track WHERE GenreId = (SELECT GenreId FROM Genre WHERE Name = 'Rock');",
    },
    {
        "input": "Find the total duration of all tracks.",
        "query": "SELECT SUM(Milliseconds) FROM Track;",
    },
    {
        "input": "List all customers from Canada.",
        "query": "SELECT * FROM Customer WHERE Country = 'Canada';",
    },
    {
        "input": "How many tracks are there in the album with ID 5?",
        "query": "SELECT COUNT(*) FROM Track WHERE AlbumId = 5;",
    },
    {
        "input": "Find the total number of invoices.",
        "query": "SELECT COUNT(*) FROM Invoice;",
    },
    {
        "input": "List all tracks that are longer than 5 minutes.",
        "query": "SELECT * FROM Track WHERE Milliseconds > 300000;",
    },
    {
        "input": "Who are the top 5 customers by total purchase?",
        "query": "SELECT CustomerId, SUM(Total) AS TotalPurchase FROM Invoice GROUP BY CustomerId ORDER BY TotalPurchase DESC LIMIT 5;",
    },
    {
        "input": "Which albums are from the year 2000?",
        "query": "SELECT * FROM Album WHERE strftime('%Y', ReleaseDate) = '2000';",
    },
    {
        "input": "How many employees are there",
        "query": 'SELECT COUNT(*) FROM "Employee"',
    },
]

#### Example Selector

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

e:\Repos\IA\NLP-SQL\nl-sql-env\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    hf,
    FAISS,
    k=5,
    input_keys=["input"],
)

#### Mejorando el Prompt

In [14]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
    SystemMessagePromptTemplate,
)

system_prefix = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer.

Here are some examples of user inputs and their corresponding SQL queries:"""

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=PromptTemplate.from_template(
        "User input: {input}\nSQL query: {query}"
    ),
    input_variables=["input", "dialect", "top_k"],
    prefix=system_prefix,
    suffix="",
)

In [15]:
full_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=few_shot_prompt),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [16]:
prompt_val = full_prompt.invoke(
    {
        "input": "How many artists are there",
        "top_k": 5,
        "dialect": "SQLite",
        "agent_scratchpad": [],
    }
)
print(prompt_val.to_string())

System: You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't 

In [17]:
agent = create_sql_agent(
    llm=llm,
    db=db,
    prompt=full_prompt,
    verbose=True,
    agent_type="openai-tools",
)

In [18]:
response = agent.invoke({"input": "todos los artistas y el numero total de canciones que poseen"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Artist, Track'}`



CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Track" (
	"TrackId" INTEGER NOT NULL, 
	"Name" NVARCHAR(200) NOT NULL, 
	"AlbumId" INTEGER, 
	"MediaTypeId" INTEGER NOT NULL, 
	"GenreId" INTEGER, 
	"Composer" NVARCHAR(220), 
	"Milliseconds" INTEGER NOT NULL, 
	"Bytes" INTEGER, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("TrackId"), 
	FOREIGN KEY("MediaTypeId") REFERENCES "MediaType" ("MediaTypeId"), 
	FOREIGN KEY("GenreId") REFERENCES "Genre" ("GenreId"), 
	FOREIGN KEY("AlbumId") REFERENCES "Album" ("AlbumId")
)

/*
3 rows from Track table:
TrackId	Name	

Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ResourceExhausted: 429 Online prediction request quota exceeded for gemini-1.5-pro. Please try again later with backoff..
Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ResourceExhausted: 429 Online prediction request quota exceeded for gemini-1.5-pro. Please try again later with backoff..


A continuacion se muestra el nombre del artista y el numero total de canciones de cada artista:
AC/DC: 18
Aaron Copland & London Symphony Orchestra: 1
Aaron Goldberg: 1
Academy of St. Martin in the Fields & Sir Neville Marriner: 2
Academy of St. Martin in the Fields Chamber Ensemble & Sir Neville Marriner: 1
Academy of St. Martin in the Fields, John Birch, Sir Neville Marriner & Sylvia McNair: 1
Academy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart: 1
Accept: 4
Adrian Leaper & Doreen de Feis: 1
Aerosmith: 15
... (truncated due to character limits)


> Finished chain.


In [20]:
response.get('output')

'A continuacion se muestra el nombre del artista y el numero total de canciones de cada artista:\nAC/DC: 18\nAaron Copland & London Symphony Orchestra: 1\nAaron Goldberg: 1\nAcademy of St. Martin in the Fields & Sir Neville Marriner: 2\nAcademy of St. Martin in the Fields Chamber Ensemble & Sir Neville Marriner: 1\nAcademy of St. Martin in the Fields, John Birch, Sir Neville Marriner & Sylvia McNair: 1\nAcademy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart: 1\nAccept: 4\nAdrian Leaper & Doreen de Feis: 1\nAerosmith: 15\n... (truncated due to character limits)\n'

In [24]:
import markdown
f_response = markdown.markdown(response.get('output'))
print(f_response)

<p>A continuacion se muestra el nombre del artista y el numero total de canciones de cada artista:
AC/DC: 18
Aaron Copland &amp; London Symphony Orchestra: 1
Aaron Goldberg: 1
Academy of St. Martin in the Fields &amp; Sir Neville Marriner: 2
Academy of St. Martin in the Fields Chamber Ensemble &amp; Sir Neville Marriner: 1
Academy of St. Martin in the Fields, John Birch, Sir Neville Marriner &amp; Sylvia McNair: 1
Academy of St. Martin in the Fields, Sir Neville Marriner &amp; Thurston Dart: 1
Accept: 4
Adrian Leaper &amp; Doreen de Feis: 1
Aerosmith: 15
... (truncated due to character limits)</p>


In [22]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [2]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

index = faiss.IndexFlatL2(len(hf.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=hf,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)